In [1]:
from poseidon import poseidon
import time
import hashlib
import pandas as pd
import random
import argparse
import json
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn
from eapdataset import Dataset
    
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")


#print(par1)
#par1='abcde0001'

#%run train222.py --masterid $par1 --datasetname $datasetname  
    
# Model Tranining Sample
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


par1=os.environ.get("--masterid")
par2=os.environ.get("--datasetname")
#par2='power-forecast'

dataset = Dataset.get_by_name(par2)
df2=dataset.to_pandas_dataframe(use_cached_result=False)

np.random.seed(40)
data={}
try:
    data=df2.head(24962).fillna('1')

except Exception as e:
    logger.exception(
            "Get Hive Data Error: %s", e)

    
train, test = train_test_split(data)

# The predicted column is "power"
train_x = train.drop(["x_basic_forecast_time","x_basic_time","ec_nwp_time","gfs_nwp_time","power","speed"], axis=1)
test_x = test.drop(["x_basic_forecast_time","x_basic_time","ec_nwp_time","gfs_nwp_time","power","speed"], axis=1)
train_y = train[["power"]]
test_y = test[["power"]]
    
    
# Set default values if no alpha is provided
alpha = 0.5
l1_ratio = 0.5



# Useful for multiple runs (only doing one run in this sample notebook)    
with mlflow.start_run() as run:
    # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
        
    #print(test_x)
    # Evaluate Metrics
    predicted_powers = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_powers)



    # Log parameter, metrics, and model to MLflow
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

C:\Users\jiabo.zhou\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


NameError: name 'Dataset' is not defined